In [1]:
from os import getcwd
import os.path as op
from glob import glob

In [2]:
from clients.utils import create_allocation_table
from hardware.plates import create_plates_from_allocation_table

expt_name = 'A81_E214'
allocation_table = create_allocation_table(expt_name)
all_expt_plates = create_plates_from_allocation_table(allocation_table)

In [3]:
from hardware.qpcr import QpcrDataFile

id_qpcr_plate = 'A81_E214_1_ID'
cwd = getcwd()
nav_root = op.abspath(op.join(cwd, op.pardir,
                              op.pardir, op.pardir))
data_folder = op.join(nav_root, 'hardware', 'tests', 'data')
f = op.join(data_folder, id_qpcr_plate + '.xls')

qpcr_data = QpcrDataFile(f)
raw_instrumemt_data = {id_qpcr_plate: qpcr_data.get_data_by_well()}

C:\Users\chris.icely\AppData\Local\conda\conda\envs\jupyter\lib\site-packages\pandas\core\ops.py:816: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


In [4]:
from hardware.labchip import LabChip

lc_plate = '20180103_A'
fds = glob(op.join(data_folder, '*' + lc_plate + '*.csv'))

files = {}
for f in fds:
    if 'PeakTable' in f:
        files['peak_file'] = f
    elif 'RawTable' in f:
        files['raw_file'] = f
    elif 'WellTable' in f:
        
        files['well_file'] = f
lc = LabChip(**files)
lc_data_plates = {lc_plate: lc.get_data_by_well()}

In [5]:
from clients.expt_recipes.nested.model_builders import get_id_qpcr_plate_names,\
    build_id_qpcr_constituents, build_id_qpcr_datas

from clients.expt_recipes.nested.model_builders import (get_labchip_plate_names,
                                                        create_id_qpcr_lc_mapping,
                                                        get_lc_dilutions)
from clients.expt_recipes.nested.model_builders import get_lc_dilutions
from clients.expt_recipes.nested.model_builders import build_labchip_results_by_id_well
    
assays = {'Ca_rpb7_x.1_Cal04_Cal03': 215,
          'Cg_rps0_x.1_Cgl03_Cgl04': 217,
          'Ab_pgaD_x.10_Aba38_Aba42': 225,
          'Ec_uidA_x.2_Eco64_Eco66': 240}
    
id_qpcr_plates = get_id_qpcr_plate_names(all_expt_plates)
lc_plates = get_labchip_plate_names(all_expt_plates)

id_plate_constituents = {}
id_plate_results = {}
downstream_id_lc_results = {}
for idp in id_qpcr_plates:
    id_plate_constituents[idp] = build_id_qpcr_constituents(all_expt_plates[idp], all_expt_plates)
    id_plate_results[idp] = build_id_qpcr_datas(id_plate_constituents[idp],
                                             raw_instrumemt_data[idp])
    for lcp in lc_plates:
        mapping = create_id_qpcr_lc_mapping(all_expt_plates[lcp], lcp)
        dilutions = get_lc_dilutions(all_expt_plates[lcp])
        downstream_id_lc_results[idp] = build_labchip_results_by_id_well(
            id_plate_constituents[idp], lc_data_plates[lcp],
            mapping[idp], assays, dilutions)

In [6]:
id_plate_constituents[id_qpcr_plate]['A01']

NestedIdWellConstituents([('assays',
                           [{'concentration': 0.4,
                             'reagent_category': 'assay',
                             'reagent_name': 'Ca_rpb7_x.1_Cal04_Cal03',
                             'unit': 'uM'}]),
                          ('transferred_assays',
                           [{'concentration': 1.0,
                             'reagent_category': 'group assay',
                             'reagent_name': 'poolE1',
                             'unit': 'x'}]),
                          ('templates', []),
                          ('transferred_templates', []),
                          ('human', []),
                          ('transferred_human', [])])

In [7]:
id_plate_results[id_qpcr_plate]['A01']

NestedIdQpcrData([('Ct', 5.587069988250732),
                  ('∆NTC_Ct', 0.0),
                  ('Ct_Call', 'NEG'),
                  ('Tm1', 79.1648941040039),
                  ('Tm2', 74.78462219238281),
                  ('Tm3', 62.46510314941406),
                  ('Tm4', nan),
                  ('Tm Specif', False),
                  ('Tm NS', False),
                  ('Tm PD', True)])

In [8]:
downstream_id_lc_results[id_qpcr_plate]['A01']

NestedLabChipData([('Specif ng/ul', 0),
                   ('NS ng/ul', 0),
                   ('PD ng/ul', 11.519988986198639)])